# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [145]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to sdhar.bigdata@gmail.com and will expire on April 14, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Sumit\AppData\Local\Temp\graphlab_server_1493847871.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Some useful functions

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [36]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.292084     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 0.317101     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [37]:
print 'number of nonzeros = %d' % (model_all.coefficients['value']).nnz()
cf =  model_all.coefficients
cf.print_rows(20)
print '\n'

number of nonzeros = 6
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sq

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [38]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [43]:
l1_penalty_rss = []
min_rss = None
best_l1_penalty = None
for l1 in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1, verbose = False)
    next_rss =((validation['price']- model.predict(validation))**2).sum()
    #print next_rss
    l1_penalty_rss.append(next_rss)
    if min_rss is None or next_rss < min_rss:
            min_rss = next_rss
            best_l1_penalty = l1
print min_rss, best_l1_penalty

6.25766285142e+14 10.0


In [44]:
print best_l1_penalty

10.0


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [46]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty= best_l1_penalty, verbose = False)
print 'number of nonzeros = %d' % (model.coefficients['value']).nnz()
cf =  model.coefficients
cf.print_rows(20)


number of nonzeros = 18
+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sq

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [122]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [49]:
l1_penalty_values = np.logspace(8, 10, num=20)

print l1_penalty_values

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [99]:
coef_dict = {}
l1_list = []
for l1 in np.logspace(8, 10, num=20):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1, verbose = False)
    next_rss =((validation['price']- model.predict(validation))**2).sum()
    coef_dict[l1] = model['coefficients']['value'].nnz()

In [101]:
import pprint
pprint.pprint(coef_dict)

{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.11147901: 18,
 263665089.87303555: 17,
 335981828.62837881: 17,
 428133239.8719396: 17,
 545559478.11685145: 17,
 695192796.17755914: 17,
 885866790.41008317: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313128: 10,
 3792690190.7322536: 6,
 4832930238.5717525: 5,
 6158482110.6602545: 3,
 7847599703.5146227: 1,
 10000000000.0: 1}


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [ ]:
l1_penalty_min = 2976351441.6313128
l1_penalty_max = 3792690190.7322536

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [102]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
print l1_penalty_values

[  2.97635144e+09   3.34601716e+09   3.71568287e+09   4.08534858e+09
   4.45501430e+09   4.82468001e+09   5.19434572e+09   5.56401144e+09
   5.93367715e+09   6.30334286e+09   6.67300858e+09   7.04267429e+09
   7.41234000e+09   7.78200572e+09   8.15167143e+09   8.52133715e+09
   8.89100286e+09   9.26066857e+09   9.63033429e+09   1.00000000e+10]


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [138]:
val_rss = {}
best_l1 = 0
best_rss= 1e+25
for l1 in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1, verbose = False)
    rss =((validation['price']- model.predict(validation))**2).sum()
    spars = model['coefficients']['value'].nnz()
    if (spars <= max_nonzeros) and (best_rss > rss):
        best_l1 = l1
        best_rss = rss
        best_spars = spars
    val_rss[l1] = rss, model['coefficients']['value'].nnz()


print 'Best L1:', best_l1, 'Best RSS %e' % best_rss, 'Sparsity:', best_spars

Best L1: 3715682868.83 Best RSS 1.074360e+15 Sparsity: 6


In [139]:
validation_rss = {}
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    residuals = validation['price'] - predictions
    rss = sum(residuals**2)
    validation_rss[l1_penalty] = rss, model['coefficients']['value'].nnz()
    
for k,v in validation_rss.iteritems():    
    if (v[1] == max_nonzeros) and (v[0] < bestRSS):
        bestRSS = v[0]
        bestl1 = k
        
print bestRSS, bestl1

1.07436022541e+15 3715682868.83


In [141]:
for k,v in validation_rss.iteritems():
    if (v[1] == max_nonzeros) and (v[0] < bestRSS):
        bestRSS = v[0]
print k, bestRSS

5194345723.22 1.07436022541e+15


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [144]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=3715682868.83)
non_zero_weight_test = model["coefficients"][model["coefficients"]["value"] > 0]
non_zero_weight_test.print_rows(num_rows=8)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 236298.089788 |  None  |
|    bathrooms     |  None | 14318.5263999 |  None  |
|   sqft_living    |  None |  31.002043362 |  None  |
| sqft_living_sqrt |  None | 615.241574365 |  None  |
|      grade       |  None |  2412.666806  |  None  |
|    sqft_above    |  None | 28.0711724548 |  None  |
+------------------+-------+---------------+--------+
[6 rows x 4 columns]



In [143]:
model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=3448968612.16)
non_zero_weight_test = model["coefficients"][model["coefficients"]["value"] > 0]
non_zero_weight_test.print_rows(num_rows=8)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
+------------------+-------+---------------+--------+
[7 rows x 4 columns]

